In [ ]:
def old_predictTimeSeriesNew(file_path, sliding_window=1, threshold=0.1, asPercentile=True, percentile=75.0, gamma='scale', kernel='rbf', epsilon=0.1, C=1.0):
    """
    This function reads in csv's in the given path and processes each of them, based on the sliding window.
    Creates a Support Vector Regression Model and fits 80% of the data and predicts 20% of the data
    
    file_path - path to the csv files
    sliding_window - how many previous entries should be considered as inputs to the current entry
    
    return - dictionary of precision, recall and F1-scores for each csv file
    """

    dict = {}
    a3_csv = glob.glob(file_path, recursive=True)
    for index,file in enumerate(a3_csv):

        fname = file.split("/")[4].replace('\\','').split(".")[0]
        
        if(not(fname == 'A3Benchmark_all' or fname == 'A4Benchmark_all')):
        
            #print("File name {}".format(fname))
        
            # Read in 1 csv file
            yahoo_df = pd.read_csv(file)


            # these csv files have 9 attributes, but we need only 
            yahoo_df = yahoo_df[['timestamps','value','anomaly']]

            # Extract column names
            columnNames = list(yahoo_df.columns)

            # Make a copy of the dataframe, one will be used as output, and the other as input
            yahoo_df_copy = yahoo_df.copy(deep=True)

            # first append a NaN row to the dataframe, because the last row will be lost when shifted
            yahoo_df_copy = yahoo_df_copy.append(pd.Series(), ignore_index=True)
            yahoo_df_copy = yahoo_df_copy.shift(1)

        #     print("Copy head")
        #     print(yahoo_df_copy.head(5))

            # a NaN row is required to be able to merge
            yahoo_df = yahoo_df.append(pd.Series(), ignore_index=True)

        #     print("Original head")
        #     print(yahoo_df.head(5))

            yahoo_df = yahoo_df.rename(columns=createColumnDict(columnNames))

            yahoo_merged = yahoo_df_copy.merge(yahoo_df, left_index=True, right_index=True)

        #     print("Merged head")
        #     print(yahoo_merged.head(5))

        #     print("Merged tail")
        #     print(yahoo_merged.tail(5))

        #     print(yahoo_merged.shape)
        #     print(yahoo_merged.head(0))

            # TODO put this in a for loop for sliding_window > 1
            last_index = yahoo_merged.shape[0] - 1
            # drop the 1st and last rows, because they contain NaN values
            yahoo_merged = yahoo_merged.drop([0,last_index])

    #         yahoo_merged = yahoo_merged.drop(['is_anomaly','is_anomaly_y','timestamp', 'timestamp_y'], axis=1)
            train_size = int(len(yahoo_merged) * 0.8)
            train_set, test_set = yahoo_merged[:train_size], yahoo_merged[train_size:]
            #seperate into features and target
            X_train = train_set[['value']]
            y_train = train_set[['value_y']]
            X_test = test_set[['value']]
            y_test = test_set[['value_y']]

            # outlier_y column is not needed for forecasting, but needed later to detect outliers
            outlier_df = test_set[['anomaly_y']]

            svm_clf = SVR(kernel=kernel, gamma=gamma, C=C, epsilon=epsilon)
            svm_clf.fit(X_train, np.ravel(y_train))
            y_predict = svm_clf.predict(X_test)
    #         y_predict1 = y_predict.reshape(-1,1)

    #         print("Actual anomaly : ")
    #         print(outlier_df)

            predicted_anomaly = None
            if(asPercentile):
                predicted_anomaly = get_anomaly_labels_by_deviation_pctile_threshold(np.ravel(y_test), np.ravel(y_predict), percentile) 
            else:
                predicted_anomaly = get_anomaly_labels_by_deviation_threshold(np.ravel(y_test), np.ravel(y_predict), threshold)

            metrics = precision_recall_fscore_support(np.ravel(outlier_df), predicted_anomaly, average='binary', zero_division=0)

            dict[fname] = [metrics[0], metrics[1], metrics[2]]

    return dict



In [ ]:
def generate_median_table(p1, p2, p3, r1, r2, r3, f1, f2, f3, p1_n, p2_n, p3_n, r1_n, r2_n, r3_n, f1_n, f2_n, f3_n):
    """
    Generates 2 tables which includes the median of precision, recall and f1-score results
    First 9 lists belong in 1 table and the other 9 lists belong the other table
    
    """
    
    l1 = [['precision', statistics.median(p1), statistics.median(p2), statistics.median(p3)],
          ['recall', statistics.median(r1), statistics.median(r2), statistics.median(r3)],
          ['f1-score', statistics.median(f1), statistics.median(f2), statistics.median(f3)]]
    table1 = tabulate(l1, headers=['', 'C=1', 'C=25', 'C=50' ], tablefmt='orgtbl')
    print()
    print('Median when SVR has epsilon="0.1"\n')
    print(table1)
    
    l2 = [['precision', statistics.median(p1_n), statistics.median(p2_n), statistics.median(p3_n)],
          ['recall', statistics.median(r1_n), statistics.median(r2_n), statistics.median(r3_n)],
          ['f1-score', statistics.median(f1_n), statistics.median(f2_n), statistics.median(f3_n)]]
    table2 = tabulate(l2, headers=['', 'C=1', 'C=25', 'C=50' ], tablefmt='orgtbl')
    print()
    print('Median when SVR has epsilon="0.5"\n')
    print(table2)

In [ ]:
def draw_box_plots(p1, p2, p3, r1, r2, r3, f1, f2, f3, p1_n, p2_n, p3_n, r1_n, r2_n, r3_n, f1_n, f2_n, f3_n):

    """
    Takes in 18 lists (6 precision results lists, 6 recall results lists, 6 f1-score results lists) and plots
    6 boxplots, each having 3 box plots
    """
    
    precision_data = [p1, p2, p3] 
    recall_data = [r1, r2, r3] 
    f1_data = [f1, f2, f3] 

    new_precision_data = [p1_n, p2_n, p3_n]
    new_recall_data = [r1_n, r2_n, r3_n]
    new_f1_data = [f1_n, f2_n, f3_n]

    fig, ax = plt.subplots(3, 2, figsize=(10, 10))
    fig.subplots_adjust(hspace=0.4, wspace=0.7)

    ax[0][0].set_title('SVR when epsilon="0.1"')
    ax[0][0].set_xlabel('C')
    ax[0][0].set_ylabel('Precision')
    ax[0][0].set_xticklabels([1,25,50])
    ax[0][0].boxplot(precision_data)

    ax[0][1].set_title('SVR when epsilon="0.5"')
    ax[0][1].set_xlabel('C')
    ax[0][1].set_ylabel('Precision')
    ax[0][1].set_xticklabels([1,25,50])
    ax[0][1].sharey(ax[0, 0])
    ax[0][1].boxplot(new_precision_data)

    ax[1][0].set_xlabel('C')
    ax[1][0].set_ylabel('Recall')
    ax[1][0].set_xticklabels([1,25,50])
    ax[1][0].boxplot(recall_data)
    
    ax[1][1].set_xlabel('C')
    ax[1][1].set_ylabel('Recall')
    ax[1][1].set_xticklabels([1,25,50])
    ax[1][1].sharey(ax[1, 0])
    ax[1][1].boxplot(new_recall_data)

    ax[2][0].set_xlabel('C')
    ax[2][0].set_ylabel('F1-score')
    ax[2][0].set_xticklabels([1,25,50]) 
    ax[2][0].boxplot(f1_data)
    
    ax[2][1].set_xlabel('C')
    ax[2][1].set_ylabel('F1-score')
    ax[2][1].set_xticklabels([1,25,50]) 
    ax[2][1].sharey(ax[2, 0])
    ax[2][1].boxplot(new_f1_data)

    plt.show()

In [ ]:
def getMetricResults(dictionary):
    """
    Reads data from the dictionary and generates 3 lists of precision, recall and f1-scores
    
    dictionary - dictionary containing key, value pairs of filename and metric list
    
    returns 3 lists of each metric values
    """

    precision_results = []
    recall_results = []
    f1_results = []
    for fname in dictionary:
        all_results = dictionary[fname]
        precision_results.append(all_results[0])
        recall_results.append(all_results[1])
        f1_results.append(all_results[2])
    
    return precision_results, recall_results, f1_results
    